In [1]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import torch
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from utils import datasets, utils
import os
import time

from methods.digits import ours_beta

In [2]:
x_train, x_test, y_train, y_test, classes = datasets.load('domainnet',
                                                          imbalance=datasets.IMBALANCE_AMOUNTS_DOMAINNET_DIFFERENT_PY)

domains = x_train.keys()

In [3]:
for domain in domains:
    print(domain, 'train', x_train[domain].shape)
    print(np.unique(y_train[domain], return_counts=True))
    print(domain, 'test', x_test[domain].shape)
    print(np.unique(y_test[domain], return_counts=True))
    print()

clipart train (900, 3, 32, 32)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
      dtype=int32), array([120, 120, 120, 120,  80,  60,  60,  40,  40,  25,  25,  25,  25,
        10,  10,  10,  10]))
clipart test (1020, 3, 32, 32)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
      dtype=int32), array([60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60]))

infograph train (900, 3, 32, 32)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
      dtype=int32), array([ 60,  25,  40, 120, 120,  25,  60,  40, 120,  25,  10,  80,  25,
        10,  10, 120,  10]))
infograph test (1020, 3, 32, 32)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
      dtype=int32), array([60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60]))

painting train (900, 3, 32, 32)
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
      dtype=int32

## Parameters

In [4]:
domain_t = 'quickdraw'

lr = 1e-4
batch_size = 50
epochs = 200
m_params = (epochs, batch_size, lr, 3, len(classes))
evaluation_metrics = ['acc', 'bal_acc', 'auc', 'f1']

## Select source domains

In [5]:
x_sources, y_sources = {}, {}
for domain_s in domains:
    if domain_s != domain_t:
        x_sources[domain_s] = x_train[domain_s]
        y_sources[domain_s] = y_train[domain_s]

## Method execution

In [6]:
## Instantiate method
method = ours_beta.Method(*m_params)

## Train method
method.train(x_sources, y_sources, x_train[domain_t], y_train[domain_t], domain_t=domain_t)

## Obtain predictions from trained method on target domain test set
y_probas = method.predict(x_test[domain_t])

## Eval results given evaluation metrics
met = utils.eval_results(y_probas, y_test[domain_t], evaluation_metrics)

Epoch 1/200 - Time elapsed 00:00:05
Epoch 10/200 - Time elapsed 00:00:37
Epoch 20/200 - Time elapsed 00:01:06
Epoch 30/200 - Time elapsed 00:01:39
Epoch 40/200 - Time elapsed 00:02:11
Epoch 50/200 - Time elapsed 00:02:44
Epoch 60/200 - Time elapsed 00:03:15
Epoch 70/200 - Time elapsed 00:03:46
Epoch 80/200 - Time elapsed 00:04:18
Epoch 90/200 - Time elapsed 00:04:50
Epoch 100/200 - Time elapsed 00:05:21
Epoch 110/200 - Time elapsed 00:05:53
Epoch 120/200 - Time elapsed 00:06:25
Epoch 130/200 - Time elapsed 00:06:57
Epoch 140/200 - Time elapsed 00:07:27
Epoch 150/200 - Time elapsed 00:07:55
Epoch 160/200 - Time elapsed 00:08:23
Epoch 170/200 - Time elapsed 00:08:51
Epoch 180/200 - Time elapsed 00:09:19
Epoch 190/200 - Time elapsed 00:09:48
Epoch 200/200 - Time elapsed 00:10:15


In [7]:
met

{'acc': 39.01960784313726,
 'bal_acc': 39.01960784313725,
 'auc': 86.35008169934642,
 'f1': 33.81203171253332}